In [1]:
import pickle
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from time import time
from keras.preprocessing import image

C:\Users\EXTREME_RED\AppData\Local\Temp\ipykernel_19720\4048311190.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from keras.models import Model

In [12]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [27]:
import json

def read_json_file(file_path):
    dataset = []
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            if not isinstance(data, list):
                print("Invalid JSON format: Expected a list.")
                return []
            dataset = ['VG_100K\\{}.jpg'.format(str(num)) for num in data]
    except FileNotFoundError:
        print("File not found.")
    except json.JSONDecodeError:
        print("Invalid JSON format in file.")
    except Exception as e:
        print("An error occurred:", e)
    return dataset

# Example usage:
file_path = 'train_split.json'  # Specify the path to your JSON file
train_images = read_json_file(file_path)

# Print the first 10 elements
print(train_images[:10])
print('Dataset: %d' % len(train_images))

['VG_100K\\2406949.jpg', 'VG_100K\\2387215.jpg', 'VG_100K\\2372347.jpg', 'VG_100K\\2369202.jpg', 'VG_100K\\2368287.jpg', 'VG_100K\\2352527.jpg', 'VG_100K\\2358049.jpg', 'VG_100K\\2380206.jpg', 'VG_100K\\2375670.jpg', 'VG_100K\\2389065.jpg']
Dataset: 14575


In [28]:
file_path = 'test_split.json'  # Specify the path to your JSON file
test_images = read_json_file(file_path)

# Print the first 10 elements
print(test_images[:10])
print('Dataset: %d' % len(test_images))

['VG_100K\\2411943.jpg', 'VG_100K\\2365509.jpg', 'VG_100K\\2345585.jpg', 'VG_100K\\2369883.jpg', 'VG_100K\\2365141.jpg', 'VG_100K\\2375853.jpg', 'VG_100K\\2380805.jpg', 'VG_100K\\2373168.jpg', 'VG_100K\\2339963.jpg', 'VG_100K\\2401026.jpg']
Dataset: 2489


In [39]:
file_path = 'val_split.json'  # Specify the path to your JSON file
val_images = read_json_file(file_path)

# Print the first 10 elements
print(val_images[:10])
print('Dataset: %d' % len(val_images))

['VG_100K\\2354019.jpg', 'VG_100K\\2379480.jpg', 'VG_100K\\2397756.jpg', 'VG_100K\\2375161.jpg', 'VG_100K\\2327911.jpg', 'VG_100K\\2388564.jpg', 'VG_100K\\2410083.jpg', 'VG_100K\\2408454.jpg', 'VG_100K\\2400488.jpg', 'VG_100K\\2368358.jpg']
Dataset: 2487


In [29]:
images = 'VG_100K/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [22]:
'''
# Below file conatains the names of images to be used in train data
#train_images_file = 'Flickr_8k.trainImages.txt'
# Read the train image names in a set
#train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images
'''

In [30]:
#train_img = train_images + '.jpg'
print(img[:10])

['VG_100K\\1.jpg', 'VG_100K\\10.jpg', 'VG_100K\\100.jpg', 'VG_100K\\1000.jpg', 'VG_100K\\1001.jpg', 'VG_100K\\1002.jpg', 'VG_100K\\1003.jpg', 'VG_100K\\1004.jpg', 'VG_100K\\1005.jpg', 'VG_100K\\1006.jpg']


In [9]:
'''
# Below file conatains the names of images to be used in test data
#test_images_file = 'Flickr_8k.testImages.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = read_json_file('test_split.json')

# Print the first 10 elements
print(test_images[:10])
print('Dataset: %d' % len(test_images))
#test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images
'''

[2411943, 2365509, 2345585, 2369883, 2365141, 2375853, 2380805, 2373168, 2339963, 2401026]
Dataset: 2489


In [31]:
def preprocess(image_path):
    
    img = image.load_img(image_path, target_size=(224, 224))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [11]:
model=VGG19()
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
print(model.summary())



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [32]:
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [33]:
start = time()
encoding_train = {}
for img in train_images:
    encoding_train[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

1/1 [==============================] - 0s 71ms/step
Time taken in seconds = 1602.6148929595947


In [35]:
start = time()
encoding_test = {}
for img in test_images:
    encoding_test[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

1/1 [==============================] - 0s 70ms/step
Time taken in seconds = 270.2159934043884


In [40]:
start = time()
encoding_val = {}
for img in val_images:
    encoding_val[img[len(images):]] = encode(img)
print("Time taken in seconds =", time()-start)

1/1 [==============================] - 0s 84ms/step
Time taken in seconds = 272.5292856693268


In [34]:
import pickle
with open("encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [36]:
with open("encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)

In [41]:
with open("encoded_val_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_val, encoded_pickle)

In [37]:
print(encoding_test['2411943.jpg'].shape)

(4096,)
